In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import torch
from PIL import Image
import pandas as pd

# Import or install Sionna
try:
    import sionna.rt
except ImportError as e:
    import os
    os.system("pip install sionna-rt")
    import sionna.rt

no_preview = False # Toggle to False to use the preview widget
                  # instead of rendering for scene visualization

from sionna.rt import load_scene, PlanarArray, Transmitter, Receiver, Camera,\
                      RadioMapSolver, PathSolver,transform_mesh
from sionna.rt.utils import r_hat, subcarrier_frequencies

In [2]:
scene = load_scene("Hong Hum_256.xml",
                   merge_shapes=False)


# Configure a transmitter that is located at the front of "car_2"
scene.add(Transmitter("tx", position=[6,18,1.5], orientation=[np.pi,0,0]))
scene.tx_array = PlanarArray(num_rows=1, num_cols=1, pattern="tr38901", polarization="V")
scene.rx_array = scene.tx_array

# Create radio map solver
rm_solver = RadioMapSolver()
                # rm_show_color_bar=True,
# rm = rm_solver(scene=scene,
#                    samples_per_tx=20**6,
#                    refraction=True,
#                    max_depth=10,
#                    cell_size=[2,2])
# cam =  Camera(position=[-14,35,450], look_at=[-14,35,0])
# # scene.preview(radio_map=rm,
# #                 rm_vmax=-40, rm_vmin=-150)
# scene.render_to_file(camera=cam,  filename="HH.png", radio_map=rm,
#                      resolution=(256,256),
#                 num_samples=1024,
#                 rm_vmin=-140)


In [3]:
def gray_generate(radio_map,metric,file_name,db_scale: bool = True):
    
    rm_real = radio_map.path_gain.numpy().squeeze(axis=0)
    if metric=="rss" and db_scale:
        rm_values *= 1000
    valid = np.logical_and(rm_real > 0., np.isfinite(rm_real))
    opacity = valid.astype(np.float32)
    any_valid = np.any(valid)
    rm_real[valid] = 10. * np.log10(rm_real[valid])

    vmin = rm_real[valid].min() if any_valid else 0

    vmax = rm_real[valid].max() if any_valid else 0
    normalizer = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
    color_map = matplotlib.colormaps.get_cmap('gray')
    texture = color_map(normalizer(rm_real))
    # Eliminate alpha channel
    texture = texture[..., :3]
    # Colors from the color map are gamma-compressed, go back to linear
    texture = np.power(texture, 2.2)
     # Pre-multiply alpha to avoid fringe
    texture *= opacity[..., None]

    texture_uint8 = (texture * 255).astype(np.uint8)
    texture_single = texture_uint8[..., 0] 
    return plt.imsave(file_name, texture_single, cmap='gray')

In [4]:
def world_to_pixel(world_points, sensor, image_width, image_height):
    """
    将3D世界坐标转换为2D像素坐标
    :param world_points: 形状为 (N, 3) 的numpy数组，存储3D顶点坐标 (x, y, z)
    :param sensor: Mitsuba传感器（相机）对象
    :param image_width: 渲染图片的宽度（像素）
    :param image_height: 渲染图片的高度（像素）
    :return: 形状为 (N, 2) 的numpy数组，存储像素坐标 (u, v)
    """
    # 1. 获取相机的视图变换矩阵（世界坐标 → 相机坐标）
    view_transform = sensor.world_transform()
    # 2. 获取相机的投影矩阵（相机坐标 → 归一化设备坐标NDC）
    proj_transform = sensor.projection_transform()
    
    # 3. 转换3D点为齐次坐标（添加w=1）
    homogeneous_points = np.hstack([world_points, np.ones((len(world_points), 1))])  # 形状 (N, 4)
    
    # 4. 应用视图变换（世界坐标 → 相机坐标）
    camera_points = view_transform.transform_points(homogeneous_points)  # 结果仍是齐次坐标 (N, 4)
    
    # 5. 应用投影变换（相机坐标 → NDC坐标，范围[-1,1]）
    ndc_points = proj_transform.transform_points(camera_points)  # (N, 4)
    
    # 6. 透视除法（齐次坐标转非齐次）
    ndc_x = ndc_points[:, 0] / ndc_points[:, 3]
    ndc_y = ndc_points[:, 1] / ndc_points[:, 3]
    
    # 7. NDC坐标 → 像素坐标（[-1,1] → [0, width/height]）
    # 注意：图像坐标系中y轴向下，需要翻转
    pixel_u = (ndc_x + 1) * 0.5 * image_width
    pixel_v = (1 - (ndc_y + 1) * 0.5) * image_height  # 翻转y轴
    
    # 8. 转换为整数像素坐标（可选，根据需求保留浮点或取整）
    return np.column_stack([pixel_u, pixel_v]).astype(int)


In [23]:
def get_2d_vertices(scene, rm):
    exclude_mesh_ids = set()
    exclude_mesh_ids.add(rm.measurement_surface.id())
    for i, sh in enumerate(scene.mi_scene.shapes()):
        assert sh.is_mesh()
        if exclude_mesh_ids and sh.id() in exclude_mesh_ids:
            continue
        original_id = sh.bsdf().radio_material.name #!!!
        # 获取顶点坐标缓冲区
        vertices_buffer = sh.vertex_positions_buffer()
        vertices_buffer = np.array(vertices_buffer)
        # 获取顶点坐标
        vertices_np = np.frombuffer(vertices_buffer, dtype=np.float32).reshape(-1, 3)
        print(f"网格 {sh.id()} 的顶点坐标：")
        print(f"总顶点数：{sh.vertex_count}")
        print(f"前 5 个顶点：\n{vertices_np[:5]}")  # 打印前5个顶点的3D坐标
    all_vertices = []
    for sh in scene.mi_scene.shapes():
        if not sh.is_mesh() or (exclude_mesh_ids and sh.id() in exclude_mesh_ids):
            continue
        # 获取顶点坐标（3D世界坐标）
        vertices_np = np.array(sh.vertex_positions_buffer()).reshape(-1, 3)
        all_vertices.append(vertices_np)

    # 计算全局最小y值（地面参考高度）
    all_vertices_np = np.vstack(all_vertices)
    ground_y_threshold = np.min(all_vertices_np[:, 2])  # 假设y轴是高度方向
    # 可加一个小偏差（如±0.1），避免因浮点误差漏掉地面顶点
    epsilon = 0.1
    ground_y_min = ground_y_threshold - epsilon
    ground_y_max = ground_y_threshold + epsilon

    # 筛选每个网格中的地面顶点
    ground_vertices = []  # 存储地面顶点的3D坐标 (x, y, z)
    for sh in scene.mi_scene.shapes():
        if not sh.is_mesh() or (exclude_mesh_ids and sh.id() in exclude_mesh_ids):
            continue
        vertices_np = np.array(sh.vertex_positions_buffer()).reshape(-1, 3)
        # 筛选y坐标在地面阈值范围内的顶点
        is_ground = (vertices_np[:, 2] >= ground_y_min) & (vertices_np[:, 2] <= ground_y_max)
        ground_vertices_in_mesh = vertices_np[is_ground]
        ground_vertices.append(ground_vertices_in_mesh)

    # 合并所有地面顶点
    ground_vertices_np = np.vstack(ground_vertices)
    print(f"筛选出的地面顶点数量：{len(ground_vertices_np)}")

    return ground_vertices_np

In [24]:
displacement_vec_x_p = [10,0,0]
displacement_vec_x_n = [-10,0,0]
displacement_vec_y_p = [0,10,0]
displacement_vec_y_n = [0,-10,0]
num_displacements = 2
cam =  Camera(position=[-14,35,450], look_at=[-14,35,0])
for i in range(num_displacements+1): 
    rm = rm_solver(scene=scene,
                    samples_per_tx=20**6,
                    refraction=True,
                    max_depth=10,
                    cell_size=[1,1])
    
    gray_generate(rm,metric="path_gain",file_name=f"/home/super/Zishen/RM_data_sio/RM_data/path_gain_HH_{i}.png")

    ground_vertices_np = get_2d_vertices(scene, rm)
    image_width = 256
    image_height = 256
    ground_pixel_coords = world_to_pixel(
    ground_vertices_np, 
    sensor=cam,  # 函数输入的mi.Sensor对象
    image_width=image_width, 
    image_height=image_height)

    scene.render_to_file(camera=cam,  filename=f"HH_{i}.png", radio_map=rm,
                     resolution=(256,256),
                num_samples=1024,
                rm_vmin=-140)
    j= 1
    formatted_num = f"{j:03d}"
    a = scene.get(f"mesh-car_x_p_{formatted_num}")
    b = scene.get(f"mesh-car_x_n_{formatted_num}")
    c = scene.get(f"mesh-car_y_p_{formatted_num}")
    d = scene.get(f"mesh-car_y_n_{formatted_num}")
    while a is not None or b is not None or c is not None or d is not None:

        # Compute and render a coverage map at 0.5m above the ground
        

        j += 1
        formatted_num = f"{j:03d}"
        a = scene.get(f"mesh-car_x_p_{formatted_num}")
        b = scene.get(f"mesh-car_x_n_{formatted_num}")
        c = scene.get(f"mesh-car_y_p_{formatted_num}")
        d = scene.get(f"mesh-car_y_n_{formatted_num}")

        if a != None:
            scene.get(f"mesh-car_x_p_{formatted_num}").position += displacement_vec_x_p
        if b != None:
            scene.get(f"mesh-car_x_n_{formatted_num}").position += displacement_vec_x_n
        if c != None:
            scene.get(f"mesh-car_y_p_{formatted_num}").position += displacement_vec_y_p
        if d != None:
            scene.get(f"mesh-car_y_n_{formatted_num}").position += displacement_vec_y_n

网格 mesh-Berkeley_Building-itu_brick 的顶点坐标：
总顶点数：<nanobind.nb_bound_method object at 0x763c292d7880>
前 5 个顶点：
[[-3.9102669e+01  1.1726513e+02  5.9604645e-08]
 [-3.9160309e+01  1.0190790e+02  5.9604645e-08]
 [-3.9160309e+01  1.0190790e+02  7.1186767e+00]
 [-3.9102669e+01  1.1726513e+02  7.1186767e+00]
 [-2.5944782e+01  1.0185532e+02  5.9604645e-08]]
网格 mesh-Berkeley_Building-itu_marble 的顶点坐标：
总顶点数：<nanobind.nb_bound_method object at 0x763c292d7880>
前 5 个顶点：
[[-39.16031   101.9079      7.1186767]
 [-25.944782  101.85532     7.1186767]
 [-25.878597  117.218285    7.1186767]
 [-39.10267   117.26513     7.1186767]]
网格 mesh-Block_A-itu_brick 的顶点坐标：
总顶点数：<nanobind.nb_bound_method object at 0x763c292d7880>
前 5 个顶点：
[[6.3414745e+00 4.9775414e+01 5.9604645e-08]
 [1.8418188e+01 4.9683086e+01 5.9604645e-08]
 [1.8418188e+01 4.9683086e+01 5.1505260e+01]
 [6.3414745e+00 4.9775414e+01 5.1505260e+01]
 [1.8537956e+01 6.5484047e+01 5.9604645e-08]]
网格 mesh-Block_A-itu_marble 的顶点坐标：
总顶点数：<nanobind.nb_bound_

TypeError: 'Transform4f' object is not callable

In [ ]:
DISPLACEMENTS = {
    'x_p': [10, 0, 0],
    'x_n': [-10, 0, 0],
    'y_p': [0, 10, 0],
    'y_n': [0, -10, 0]
}
def get_cars(scene, num):
    formatted_num = f"{num:03d}"
    return {
        key: scene.get(f"mesh-car_{key}_{formatted_num}") 
        for key in DISPLACEMENTS.keys()
    }

num_displacements = 5

j = 1

cars = get_cars(scene, j)

cam =  Camera(position=[-14,35,450], look_at=[-14,35,0])

while any(car is not None for car in cars.values()):

        # Compute and render a coverage map at 0.5m above the ground
        rm = rm_solver(scene=scene,
                    samples_per_tx=20**6,
                    refraction=True,
                    max_depth=10,
                    cell_size=[2,2])

        scene.render_to_file(camera=cam,  filename=f"HH_{j}.png", radio_map=rm,
                     resolution=(256,256),
                num_samples=1024,
                rm_vmin=-140)

        
        for key, displacement in DISPLACEMENTS.items():
            car = cars[key]
            if car is not None:
                car.position += displacement

        j += 1
        cars = get_cars(scene, j)

        if j > num_displacements:
            break


In [ ]:
scene = load_scene(sionna.rt.scene.simple_street_canyon_with_cars,
                   merge_shapes=False)
cam =  Camera(position=[50,0,130], look_at=[10,0,0])


# Configure a transmitter that is located at the front of "car_2"
scene.add(Transmitter("tx", position=[22.7, 5.6, 25], orientation=[np.pi,0,0]))
scene.tx_array = PlanarArray(num_rows=1, num_cols=1, pattern="tr38901", polarization="V")
scene.rx_array = scene.tx_array

# Create radio map solver
rm_solver = RadioMapSolver()

# Move cars along straight lines for a couple of steps
displacement_vec = [10, 0, 0]
num_displacements = 2

car_materal = sionna.rt.ITURadioMaterial("car-material","metal",thickness = 0.01,color = (0.8, 0.1, 0.1))
num_cars = 1
cars = [sionna.rt.SceneObject(fname = sionna.rt.scene.low_poly_car, name = f"car{i}", radio_material=car_materal) for i in range(num_cars)]
scene.edit(add=cars)
cars[0].position  = scene.get(f"car_1").position + [2, 2, 0]

cars[0].orientation = [0, 0, 0]
for i in range(num_displacements+1):

    # Compute and render a coverage map at 0.5m above the ground
    rm = rm_solver(scene=scene,
                   samples_per_tx=20**6,
                   refraction=True,
                   max_depth=10,
                   center=[0,0,0.5],
                   orientation=[0,0,0],
                   size=[186,121],
                   cell_size=[2,2])
    # scene.render(camera=cam,  radio_map=rm,
    #              num_samples=512, resolution=(256,256),
    #              rm_show_color_bar=True,
    #              rm_vmax=-40, rm_vmin=-150)
    scene.preview(radio_map=rm,
                 resolution=(256,256),
                 rm_vmax=-40, rm_vmin=-150)
    # scene.render_to_file(camera=cam, filename = f"{i}.png", radio_map=rm,
    #              num_samples=512,resolution=(256,256),
    #              rm_vmax=-40, rm_vmin=-150)
    # # Move TX to next position
    # scene.get("tx").position -= displacement_vec

    # Move cars driving in -x direction
    for j in range(1,6):
        scene.get(f"car_{j}").position -= displacement_vec

    # Move cars driving in x direction
    for j in range(6,9):
        scene.get(f"car_{j}").position += displacement_vec

In [ ]:
# Visualize path gain
rm.show(metric="path_gain");

# Visualize received signal strength (RSS)
rm.show(metric="rss");

# Visulaize SINR
rm.show(metric="sinr");

In [ ]:
# Move the car 10m along the y-axis
car_2.position += [0, 10, 0]

# And rotate it by 90 degree around the z-axis
car_2.orientation = [np.pi/2, 0, 0]